In [1]:
print('ddd')

ddd


In [4]:
#🔹 1. CSV files
import pandas as pd

df = pd.read_csv("data.csv",sep=";")

In [5]:
df.head()

,Duration,Date,Pulse,Maxpulse,Calories
0,60,01/12/2020,110,130,409.1
1,60,02/12/2020,117,145,479.0
2,60,03/12/2020,103,135,340.0
3,45,04/12/2020,109,175,282.4
4,45,05/12/2020,117,148,406.0


In [6]:
print(df.isnull().sum())


Duration    0
Date        1
Pulse       0
Maxpulse    0
Calories    2
dtype: int64


In [ ]:
print("\n" + "="*80)
print("2️⃣  ANALYSE DE L'UNICITÉ")
print("="*80)

# Doublons complets
duplicates_all = df.duplicated().sum()
print(f"\n🔍 Doublons complets: {duplicates_all}")

if duplicates_all > 0:
    print("\n📌 Lignes dupliquées:")
    print(df[df.duplicated(keep=False)].sort_values('Date'))

# Doublons sur la date
duplicates_date = df['Date'].duplicated().sum()
print(f"\n📅 Doublons sur la colonne Date: {duplicates_date}")

if duplicates_date > 0:
    duplicate_dates = df[df['Date'].duplicated(keep=False)].sort_values('Date')
    print("\n📌 Dates dupliquées:")
    print(duplicate_dates[['Date', 'Duration', 'Pulse', 'Maxpulse', 'Calories']])



2️⃣  ANALYSE DE L'UNICITÉ

🔍 Doublons complets: 1

📌 Lignes dupliquées:
    Duration        Date  Pulse  Maxpulse  Calories
11        60  12/12/2020    100       120     250.7
12        60  12/12/2020    100       120     250.7

📅 Doublons sur la colonne Date: 1

📌 Dates dupliquées:
          Date  Duration  Pulse  Maxpulse  Calories
11  12/12/2020        60    100       120     250.7
12  12/12/2020        60    100       120     250.7

✅ Taux d'unicité: 96.88%


In [8]:
# ============================================================================
# 3. VÉRIFICATION DE LA COHÉRENCE DES RELATIONS
# ============================================================================
print("\n" + "="*80)
print("3️⃣  ANALYSE DE LA COHÉRENCE")
print("="*80)

# Conversion de la colonne Date
df['Date_parsed'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')

# Vérification: Maxpulse doit être >= Pulse
df['Coherence_Pulse'] = df['Maxpulse'] >= df['Pulse']
inconsistent_pulse = (~df['Coherence_Pulse']).sum()

print(f"\n💓 Cohérence Pulse vs Maxpulse:")
print(f"   - Incohérences détectées: {inconsistent_pulse}")

if inconsistent_pulse > 0:
    print("\n⚠️  Lignes incohérentes (Maxpulse < Pulse):")
    problematic = df[~df['Coherence_Pulse']][['Date', 'Pulse', 'Maxpulse']]
    print(problematic.to_string(index=False))

# Vérification des plages de valeurs acceptables
print("\n📊 Plages de valeurs:")
print(f"   Duration: {df['Duration'].min():.0f} - {df['Duration'].max():.0f} minutes")
print(f"   Pulse: {df['Pulse'].min():.0f} - {df['Pulse'].max():.0f} bpm")
print(f"   Maxpulse: {df['Maxpulse'].min():.0f} - {df['Maxpulse'].max():.0f} bpm")
print(f"   Calories: {df['Calories'].min():.1f} - {df['Calories'].max():.1f} kcal")

# Taux de cohérence
coherence_rate = (df['Coherence_Pulse'].sum() / len(df)) * 100
print(f"\n✅ Taux de cohérence: {coherence_rate:.2f}%")


3️⃣  ANALYSE DE LA COHÉRENCE

💓 Cohérence Pulse vs Maxpulse:
   - Incohérences détectées: 1

⚠️  Lignes incohérentes (Maxpulse < Pulse):
      Date  Pulse  Maxpulse
23/12/2020    130       101

📊 Plages de valeurs:
   Duration: 30 - 450 minutes
   Pulse: 90 - 130 bpm
   Maxpulse: 101 - 175 bpm
   Calories: 195.1 - 479.0 kcal

✅ Taux de cohérence: 96.88%


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


In [11]:
# ============================================================================
# 4. MESURE DE L'ACTUALITÉ DES DONNÉES
# ============================================================================
print("\n" + "="*80)
print("4️⃣  ANALYSE DE L'ACTUALITÉ")
print("="*80)

valid_dates = df['Date_parsed'].dropna()

if len(valid_dates) > 0:
    earliest_date = valid_dates.min()
    latest_date = valid_dates.max()
    date_range = (latest_date - earliest_date).days
    
    print(f"\n📅 Période couverte:")
    print(f"   - Date la plus ancienne: {earliest_date.strftime('%d/%m/%Y')}")
    print(f"   - Date la plus récente: {latest_date.strftime('%d/%m/%Y')}")
    print(f"   - Étendue: {date_range} jours")
    
    # Calcul de l'âge des données par rapport à aujourd'hui
    today = datetime.now()
    data_age = (today - latest_date).days
    
    print(f"\n⏰ Actualité:")
    print(f"   - Âge des données les plus récentes: {data_age} jours")
    print(f"   - Année des données: 2020")
    print(f"   - Données datant de: ~{(today.year - 2020)} ans")
    
    # Classification de l'actualité
    if data_age < 30:
        actuality = "✅ EXCELLENTE (< 1 mois)"
    elif data_age < 90:
        actuality = "🟡 BONNE (< 3 mois)"
    elif data_age < 365:
        actuality = "🟠 MOYENNE (< 1 an)"
    else:
        actuality = "🔴 FAIBLE (> 1 an)"
    
    print(f"   - Évaluation: {actuality}")


4️⃣  ANALYSE DE L'ACTUALITÉ

📅 Période couverte:
   - Date la plus ancienne: 01/12/2020
   - Date la plus récente: 31/12/2020
   - Étendue: 30 jours

⏰ Actualité:
   - Âge des données les plus récentes: 1798 jours
   - Année des données: 2020
   - Données datant de: ~5 ans
   - Évaluation: 🔴 FAIBLE (> 1 an)


In [12]:
# ============================================================================
# 5. DÉTECTION DES VALEURS ABERRANTES (EXACTITUDE)
# ============================================================================
print("\n" + "="*80)
print("5️⃣  ANALYSE DE L'EXACTITUDE (Valeurs aberrantes)")
print("="*80)

def detect_outliers_iqr(series, column_name):
    """Détecte les valeurs aberrantes avec la méthode IQR"""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    
    print(f"\n📊 {column_name}:")
    print(f"   - Q1: {Q1:.1f}, Q3: {Q3:.1f}, IQR: {IQR:.1f}")
    print(f"   - Limites acceptables: [{lower_bound:.1f}, {upper_bound:.1f}]")
    print(f"   - Valeurs aberrantes détectées: {len(outliers)}")
    
    if len(outliers) > 0:
        print(f"   - Valeurs: {outliers.tolist()}")
    
    return outliers

# Analyse pour chaque colonne numérique
outliers_duration = detect_outliers_iqr(df['Duration'].dropna(), 'Duration')
outliers_pulse = detect_outliers_iqr(df['Pulse'].dropna(), 'Pulse')
outliers_maxpulse = detect_outliers_iqr(df['Maxpulse'].dropna(), 'Maxpulse')
outliers_calories = detect_outliers_iqr(df['Calories'].dropna(), 'Calories')

# Nombre total d'anomalies
total_outliers = len(outliers_duration) + len(outliers_pulse) + len(outliers_maxpulse) + len(outliers_calories)
accuracy_rate = ((len(df) * 4 - total_outliers) / (len(df) * 4)) * 100

print(f"\n✅ Taux d'exactitude: {accuracy_rate:.2f}%")
print(f"   Total d'anomalies détectées: {total_outliers}")


5️⃣  ANALYSE DE L'EXACTITUDE (Valeurs aberrantes)

📊 Duration:
   - Q1: 60.0, Q3: 60.0, IQR: 0.0
   - Limites acceptables: [60.0, 60.0]
   - Valeurs aberrantes détectées: 8
   - Valeurs: [45, 45, 450, 30, 45, 45, 45, 45]

📊 Pulse:
   - Q1: 100.0, Q3: 106.5, IQR: 6.5
   - Limites acceptables: [90.2, 116.2]
   - Valeurs aberrantes détectées: 4
   - Valeurs: [117, 117, 90, 130]

📊 Maxpulse:
   - Q1: 120.0, Q3: 132.2, IQR: 12.2
   - Limites acceptables: [101.6, 150.6]
   - Valeurs aberrantes détectées: 2
   - Valeurs: [175, 101]

📊 Calories:
   - Q1: 250.7, Q3: 344.0, IQR: 93.3
   - Limites acceptables: [110.8, 483.9]
   - Valeurs aberrantes détectées: 0

✅ Taux d'exactitude: 89.06%
   Total d'anomalies détectées: 14
